In [63]:
import pandas as pd
import geopandas as gpd
import h3

from shapely.geometry import Polygon

### Convert Avrora locations in CSV to GeoJSON

In [75]:
def csv_to_geojson(csv_path: str, geojson_path: str) -> None:
    df = pd.read_csv(csv_path).dropna(subset=['latitude', 'longitude'])
    gdf = gpd.GeoDataFrame(
        df.drop(columns=['latitude', 'longitude']),
        geometry=gpd.points_from_xy(df.longitude, df.latitude), 
        crs='EPSG:4326'
    )
    gdf.to_file(geojson_path, driver='GeoJSON', encoding='utf-8')

In [77]:
for city in ['kyiv', 'lviv']:
    csv_to_geojson(
        f'geodata/{city}/{city}-avrora-locations.csv',
        f'geodata/{city}/{city}-avrora-locations.geojson'
    )

### Convert cities boundaries to hexagonal polygons

In [78]:
def convert_boundary_to_polygons(boundary_path: str, polygons_path: str, resolution: int) -> None:
    
    boundary_geom = gpd.read_file(boundary_path).to_crs('EPSG:4326').union_all()
    boundary_polygon = h3.LatLngPoly([(lat, lng) for lng, lat in boundary_geom.exterior.coords])
    
    city_cells = h3.polygon_to_cells(boundary_polygon, resolution)
    city_polygons = [Polygon([(lng, lat) for lat, lng in h3.cell_to_boundary(cell)]) for cell in city_cells]
    
    gdf = gpd.GeoDataFrame({'cell': list(city_cells), 'geometry': city_polygons}, crs='EPSG:4326')
    gdf.to_file(polygons_path, driver='GeoJSON', encoding='utf-8')

In [79]:
for city in ['kyiv', 'lviv']:
    for resolution in [7, 8, 9]:
        convert_boundary_to_polygons(
            f'geodata/{city}/{city}-boundary.geojson', 
            f'geodata/{city}/{city}-polygons-{resolution}.geojson', 
            resolution
        )